<a href="https://colab.research.google.com/github/uvictoli/Machine_Learning/blob/main/Lv4/Lv4_%ED%8A%9C%EB%8B%9D_4_7_LGBM_%ED%8A%9C%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [↩️ 리스트로 돌아가기](https://dacon.io/competitions/open/235698/overview/description)

# 셀을 실행할 때 팝업이 뜨면 `무시하고 계속하기` 를 눌러주세요


## 데이터 다운로드
---
아래 셀을 실행시켜 데이터를 colab 에 불러옵니다.
셀 실행은 Ctrl + Enter 를 이용해 실행시킬 수 있습니다.

In [1]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-02-18 09:40:32--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-02-18 09:40:32--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.250.157.100, 142.250.157.138, 142.250.157.102, ...
Connecting to drive.google.com (drive.google.com)|142.250.157.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/600e7m2ofq86ft6mai6slnet202d10ae/1645177200000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download [following]
--2022-02-18 09:40:33--  https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0r

In [2]:
# 라이브러리 및 데이터 불러오기

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# 데이터를 불러와 학습시킬 준비하기

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Scailing
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

# Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])

onehot = encoder.transform(test[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
test = pd.concat([test, onehot], axis = 1)
test = test.drop(columns = ['type'])

test.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Scaled fixed acidity,x0_red,x0_white
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0.429752,0.0,1.0
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0.785124,1.0,0.0
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0.223140,0.0,1.0
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0.280992,0.0,1.0
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0.247934,0.0,1.0


In [3]:
pip install bayesian-optimization

In [4]:
# Bayesian Optimization 불러오기
from bayes_opt import BayesianOptimization

## 👋 실습
---
BayesianOptimization를 이용해  다음 3개의 lgbm 파라미터를 튜닝 해봅시다.
1. `n_estimators`
2. `max_depth`
3. `subsample`


In [5]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요
X = train.drop(columns = ['index', 'quality'])
y = train['quality']

In [6]:
# LGBM의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요
## Key는 LGBM hyperparameter이름이고, value는 탐색할 범위 입니다.
lgbm_parameter_bounds = {
    'n_estimators' : (30, 100),
    'max_depth' : (1, 3),
    'subsample' : (0.5, 1)
}

In [7]:
# 함수를 만들어주겠습니다.
# 함수의 구성은 다음과 같습니다.
# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들
# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성
# 3. 그 딕셔너리를 바탕으로 모델 생성
# 4. train_test_split을 통해 데이터 train-valid 나누기
# 5. 모델 학습
# 6. 모델 성능 측정
# 7. 모델의 점수 반환
def lgbm_bo(n_estimators, max_depth, subsample):
  lgbm_params = {
      'n_estimators' : int(round(n_estimators)),
      'max_depth' : int(round(max_depth)),
      'subsample' : int(round(subsample))
  }

  lgbm = LGBMClassifier(**lgbm_params)

  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

  lgbm.fit(X_train, y_train)

  score = accuracy_score(y_valid, lgbm.predict(X_valid))
  return score

In [8]:
# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.
# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요
BO_lgbm = BayesianOptimization(
    f = lgbm_bo, pbounds = lgbm_parameter_bounds, random_state = 0
)

In [9]:
# predict 메소드와 test_one 데이터를 이용해 품질 예측

In [10]:
# Bayesian Optimization을 실행해보세요
BO_lgbm.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... | subsample |
-------------------------------------------------------------
|  1        |  0.5718   |  2.098    |  80.06    |  0.8014   |
|  2        |  0.5473   |  2.09     |  59.66    |  0.8229   |
|  3        |  0.5418   |  1.875    |  92.42    |  0.9818   |
|  4        |  0.5355   |  1.767    |  85.42    |  0.7644   |
|  5        |  0.5445   |  2.136    |  94.79    |  0.5355   |
|  6        |  0.5536   |  2.242    |  78.5     |  0.7679   |
|  7        |  0.55     |  1.325    |  80.57    |  0.5434   |
|  8        |  0.5627   |  2.023    |  80.08    |  0.855    |
|  9        |  0.5691   |  2.178    |  79.94    |  0.7617   |
|  10       |  0.5555   |  1.744    |  38.08    |  0.9286   |


In [11]:
# 튜닝된 파라미터를 바탕으로 test 데이터 셋 예측

#학습
lgbm_tune = LGBMClassifier(
    n_estimators = 43, max_depth = 3, subsample = 1
)
lgbm_tune.fit(X, y)

#예측
pred = lgbm_tune.predict(test.drop(columns = ['index']))

#정답 파일 내보내기
sub = pd.read_csv('data/sample_submission.csv')
sub['quality'] = pred
sub.to_csv('tune_lgbm.csv', index = False)

## 정답

In [12]:
# X에 학습할 데이터를, y에 목표 변수를 저장해주세요
X = train.drop(columns = ['index', 'quality'])
y = train['quality']

In [13]:
# LGBM의 하이퍼 파라미터의 범위를 dictionary 형태로 지정해주세요
## Key는 LGBM hyperparameter이름이고, value는 탐색할 범위 입니다.
lgbm_parameter_bounds = {
                      'n_estimators' : (30,100),
                      'max_depth' : (1,3), # 나무의 깊이
                      'subsample' : (0.5,1)
                      }

In [14]:
# 함수를 만들어주겠습니다.
# 함수의 구성은 다음과 같습니다.
# 1. 함수에 들어가는 인자 = 위에서 만든 함수의 key값들
# 2. 함수 속 인자를 통해 받아와 새롭게 하이퍼파라미터 딕셔너리 생성
# 3. 그 딕셔너리를 바탕으로 모델 생성
# 4. train_test_split을 통해 데이터 train-valid 나누기
# 5 .모델 학습
# 6. 모델 성능 측정
# 7. 모델의 점수 반환

def lgbm_bo(n_estimators,max_depth, subsample):
  lgbm_params = {
              'n_estimators' : int(round(n_estimators)),
              'max_depth' : int(round(max_depth)),
               'subsample' : int(round(subsample)),      
              }
  lgbm = LGBMClassifier(**lgbm_params)

  X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, )

  lgbm.fit(X_train,y_train)
  score = accuracy_score(y_valid, lgbm.predict(X_valid))
  return score

In [15]:
# 이제 Bayesian Optimization을 사용할 준비가 끝났습니다.
# "BO_lgbm"라는 변수에 Bayesian Optmization을 저장해보세요
BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds,random_state = 0)

In [16]:
# Bayesian Optimization을 실행해보세요
BO_lgbm.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... | subsample |
-------------------------------------------------------------
|  1        |  0.5545   |  2.098    |  80.06    |  0.8014   |
|  2        |  0.5555   |  2.09     |  59.66    |  0.8229   |
|  3        |  0.54     |  1.875    |  92.42    |  0.9818   |
|  4        |  0.57     |  1.767    |  85.42    |  0.7644   |
|  5        |  0.5509   |  2.136    |  94.79    |  0.5355   |
|  6        |  0.5718   |  2.874    |  84.17    |  0.7028   |
|  7        |  0.5355   |  1.804    |  30.01    |  0.678    |
|  8        |  0.5555   |  2.933    |  51.34    |  0.9423   |
|  9        |  0.5336   |  1.05     |  83.44    |  0.6119   |
|  10       |  0.5427   |  2.478    |  84.96    |  0.7083   |


In [17]:
# 튜닝된 파라미터를 바탕으로 test 데이터 셋 예측

#학습
lgbm_tune =LGBMClassifier(n_estimators = 43 ,max_depth = 3, subsample = 1)
lgbm_tune.fit(X,y)


#예측
pred = lgbm_tune.predict(test.drop(columns = ['index'] ))

#정답파일 내보내기
sub = pd.read_csv('data/sample_submission.csv')
sub['quality'] = pred
sub.to_csv('tune_lgbm.csv',index = False)

# [↩️ 리스트로 돌아가기](https://dacon.io/competitions/open/235698/overview/description)